# RAPTOR-Style Hierarchical Summarization Demo

This notebook walks through a lightweight RAPTOR workflow over the long-form text in `themen_text.txt`. We will:

1. Split the document into manageable chunks.
2. Embed the chunks with OpenAI embeddings (no vector store needed).
3. Run PCA to project embeddings to 2D and visualize them.
4. Cluster the chunks, visualize the clusters, and summarize each cluster with an LLM.
5. Embed the summaries, cluster them again, and create a "summary of summaries" (the RAPTOR root).


> **adRAP vs. klassisches RAPTOR**  
> Der aktuelle Stand der Forschung (arXiv:2410.01736) beschreibt, wie **adRAP** die aus RAPTOR bekannte rekursive Baumstruktur f?r dynamische Datens?tze erh?lt. Statt den kompletten Baum nach jedem Dokument neu zu bauen, werden neue Chunks per adaptivem UMAP/GMM nur in die betroffenen Cluster einsortiert und ausschlie?lich deren Zusammenfassungen aktualisiert. Genau diese Idee greifen wir hier auf: Das Notebook demonstriert, wie wir Chroma als Vektorspeicher nutzen, um neue Texte incremental zu persistieren, nur die relevanten Cluster neu zu abstrahieren und damit Abfragen wie _?Zu welchen Themen hast du Informationen??_ jederzeit beantworten zu k?nnen ? ohne einen vollst?ndigen Rebuild starten zu m?ssen.

## 0. Install dependencies (one-time)

Install the required packages with the following command if they are not already available in your environment.

In [ ]:
!pip install -qU \
    langchain-openai \
    langchain-text-splitters \
    langchain-core \
    openai \
    matplotlib \
    seaborn \
    scikit-learn \
    pandas \
    numpy \
    python-dotenv \
    langchain-chroma>=0.1.2 \
    chromadb


In [ ]:
from dotenv import load_dotenv
load_dotenv()


## 1. Environment & imports

Store your API keys securely via environment variables. The prompts below only run if a key is missing.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

sns.set_context("talk")

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

sns.set_context("talk")

## 2. Load the source document

In [ ]:
document_path = Path("themen_text.txt")
document = document_path.read_text(encoding="utf-8")
print(f"Loaded {document_path} with {len(document.split())} words")
document[:500]

## 3. Split into chunks

We use the requested `RecursiveCharacterTextSplitter` with `chunk_size=100` and `chunk_overlap=0`.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
chunks = text_splitter.split_text(document)
chunk_df = pd.DataFrame({"chunk_id": range(len(chunks)), "text": chunks})
chunk_df.head()

## 4. Embed chunks (no vector store)

We use `text-embedding-3-large` via `OpenAIEmbeddings`. This cell may take a while depending on the number of chunks.

In [ ]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")
chunk_vectors = np.array(embeddings_model.embed_documents(chunk_df["text"].tolist()))
chunk_vectors.shape

## 5. Dimensionality reduction with PCA

In [ ]:
pca = PCA(n_components=2, random_state=42)
chunk_coords = pca.fit_transform(chunk_vectors)
chunk_df[["pc1", "pc2"]] = chunk_coords
chunk_df.head()

### Visualization

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(chunk_df["pc1"], chunk_df["pc2"], s=30, alpha=0.7)
plt.title("Chunk embeddings projected with PCA")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

# Silhouette-Score

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def best_k_by_silhouette(X, k_min=2, k_max=15, random_state=42):
    scores = {}
    for k in range(k_min, k_max + 1):
        kmeans = KMeans(n_clusters=k, random_state=random_state, n_init="auto")
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        scores[k] = score
        print(f"k={k}: silhouette={score:.4f}")
    # bestes k zurückgeben
    best_k = max(scores, key=scores.get)
    return best_k, scores

best_k, scores = best_k_by_silhouette(chunk_vectors, k_min=2, k_max=10)
print("Bestes k nach Silhouette:", best_k)

## 6. Clustering the chunks

We keep it simple with K-Means. Adjust `n_clusters` to taste.

In [ ]:
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
chunk_df["cluster"] = kmeans.fit_predict(chunk_vectors)
chunk_df.groupby("cluster").size()

In [ ]:
plt.figure(figsize=(10, 7))
sns.scatterplot(
    data=chunk_df,
    x="pc1",
    y="pc2",
    hue="cluster",
    palette="tab10",
    s=50,
)
plt.title("Chunk clusters after PCA")
plt.legend(title="Cluster")
plt.show()

## 7. Summarize each cluster with an LLM

The RAPTOR approach compresses each cluster into a higher-level node. We feed the concatenated chunk texts to a chat model and request a concise summary.

In [ ]:
import json

def _parse_cluster_summary(raw_text: str, cluster_id: int) -> tuple[str, str]:
    """Return (topic, summary_text)."""
    text = raw_text.strip()
    try:
        data = json.loads(text)
        topic = data.get("topic") or data.get("title") or f"Cluster {cluster_id}"
        summary_text = data.get("summary") or data.get("content") or text
        return topic.strip(), summary_text.strip()
    except json.JSONDecodeError:
        # fallback: first sentence as topic
        parts = text.split("
", 1)
        topic = parts[0][:80] if parts else f"Cluster {cluster_id}"
        summary_text = text if len(parts) == 1 else parts[1]
        return topic.strip() or f"Cluster {cluster_id}", summary_text.strip()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize clusters and respond as compact JSON with keys 'topic' and 'summary'."),
    (
        "user",
        "You will receive the merged text for one cluster. Respond as JSON with keys 'topic' (<=12 words) and 'summary' (<=150 words) that captures the essential ideas.

{cluster_text}",
    ),
])

cluster_summaries = []

for cluster_id, group in chunk_df.groupby("cluster"):
    cluster_text = "

".join(group["text"].tolist())
    response = llm.invoke(prompt.format_messages(cluster_text=cluster_text))
    topic, summary_text = _parse_cluster_summary(response.content, int(cluster_id))
    cluster_summaries.append(
        {
            "cluster": int(cluster_id),
            "topic": topic,
            "summary": summary_text,
            "raw": response.content.strip(),
        }
    )

summary_df = pd.DataFrame(cluster_summaries).sort_values("cluster").reset_index(drop=True)
summary_df

In [ ]:
topic_lookup = summary_df.set_index("cluster")["topic"].to_dict()
chunk_df["topic"] = chunk_df["cluster"].map(topic_lookup)
chunk_df.head()

## 8. Embed summaries and re-cluster

Now we re-embed the cluster summaries, reduce them with PCA, and run another round of clustering.

In [ ]:
summary_vectors = np.array(embeddings_model.embed_documents(summary_df["summary"].tolist()))
summary_pca = PCA(n_components=2, random_state=42)
summary_coords = summary_pca.fit_transform(summary_vectors)
summary_df[["pc1", "pc2"]] = summary_coords

summary_clusters = min(3, len(summary_df))
summary_kmeans = KMeans(n_clusters=summary_clusters, random_state=42, n_init="auto")
summary_df["cluster_lvl2"] = summary_kmeans.fit_predict(summary_vectors)
summary_df

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=summary_df,
    x="pc1",
    y="pc2",
    hue="cluster_lvl2",
    palette="tab10",
    s=200,
)
for _, row in summary_df.iterrows():
    plt.text(row["pc1"] + 0.01, row["pc2"] + 0.01, f"C{row['cluster']}")

plt.title("Cluster summaries projected & clustered again")
plt.show()

## 9. Root summary (summary of summaries)

The final RAPTOR node combines the intermediate summaries into one high-level synopsis.

In [ ]:
root_prompt = ChatPromptTemplate.from_messages([
    ("system", "You produce an executive summary that captures every cluster."),
    (
        "user",
        "Create a single coherent summary of these cluster summaries, highlighting major themes and transitions.\n\n{summaries}",
    ),
])

root_response = llm.invoke(
    root_prompt.format_messages(
        summaries="\n\n".join(
            f"Cluster {row.cluster}: {row.summary}" for row in summary_df.itertuples()
        )
    )
)
root_summary = root_response.content.strip()
root_summary

## 10. Persist chunk embeddings in Chroma

Wir verwenden [Chroma](https://docs.trychroma.com/getting-started) als persistenten Vektorspeicher. Die Bibliothek ist quelloffen (Apache 2.0) und l?sst sich ohne weitere Credentials lokal betreiben. Optional kann Chroma Cloud genutzt werden ? dann m?ssen `CHROMA_TENANT`, `CHROMA_DATABASE` und `CHROMA_API_KEY` gesetzt werden. F?r dieses Notebook reicht jedoch der lokale Persist-Modus, sodass neu hinzugef?gte Dokumente direkt gespeichert bleiben und nicht erneut eingebettet werden m?ssen.

In [ ]:
from pathlib import Path as _Path
import shutil

persist_directory = _Path("chroma_langchain_db")
if persist_directory.exists():
    shutil.rmtree(persist_directory)

vector_store = Chroma(
    collection_name="raptor_chunks_demo",
    embedding_function=embeddings_model,
    persist_directory=str(persist_directory),
)

def _rows_to_documents(df: pd.DataFrame) -> list[Document]:
    docs = []
    for row in df.itertuples():
        docs.append(
            Document(
                page_content=row.text,
                metadata={
                    "chunk_id": int(row.chunk_id),
                    "cluster": int(row.cluster),
                    "topic": row.topic or topic_lookup.get(int(row.cluster)),
                    "source": document_path.name,
                },
            )
        )
    return docs

base_docs = _rows_to_documents(chunk_df)
base_ids = [f"chunk-{doc.metadata['chunk_id']}" for doc in base_docs]
vector_store.add_documents(documents=base_docs, ids=base_ids)
print(f"Persisted {len(base_docs)} chunks to Chroma at {persist_directory.resolve()}")

## 11. Inkrementelle Updates (adRAP-Style)

Statt den kompletten RAPTOR-Baum zu rekonstruieren, simulieren wir den in arXiv:2410.01736 beschriebenen **adRAP**-Ablauf: Neue Dokumente werden in denselben UMAP/GMM-Raum projiziert, dem wahrscheinlichsten Cluster zugewiesen, dort eingebettet und nur die betroffenen Cluster-Zusammenfassungen werden neu generiert. Der Chroma-Vektorspeicher dient dabei als langlebige Quelle f?r alle Chunks.

In [ ]:
def _summarize_cluster(cluster_id: int) -> tuple[str, str]:
    """Recompute topic + summary for a given cluster."""
    group = chunk_df[chunk_df["cluster"] == cluster_id]
    if group.empty:
        return topic_lookup.get(cluster_id, f"Cluster {cluster_id}"), ""
    cluster_text = "

".join(group["text"].tolist())
    response = llm.invoke(prompt.format_messages(cluster_text=cluster_text))
    topic, summary_text = _parse_cluster_summary(response.content, cluster_id)
    summary_df.loc[summary_df["cluster"] == cluster_id, "topic"] = topic
    summary_df.loc[summary_df["cluster"] == cluster_id, "summary"] = summary_text
    topic_lookup[cluster_id] = topic
    return topic, summary_text


def add_texts_incrementally(raw_texts: list[str], source_label: str = "manual") -> list[int]:
    """Add new documents without rebuilding the entire RAPTOR stack."""
    global chunk_df
    if not raw_texts:
        return []

    new_rows = []
    for text in raw_texts:
        for chunk_text in text_splitter.split_text(text):
            next_id = int(chunk_df["chunk_id"].max()) + 1 if not chunk_df.empty else 0
            vector = embeddings_model.embed_documents([chunk_text])[0]
            assigned_cluster = int(kmeans.predict([vector])[0])
            topic = topic_lookup.get(assigned_cluster, f"Cluster {assigned_cluster}")
            doc = Document(
                page_content=chunk_text,
                metadata={
                    "chunk_id": next_id,
                    "cluster": assigned_cluster,
                    "topic": topic,
                    "source": source_label,
                },
            )
            vector_store.add_documents(documents=[doc], ids=[f"chunk-{next_id}"])
            new_rows.append(
                {
                    "chunk_id": next_id,
                    "text": chunk_text,
                    "cluster": assigned_cluster,
                    "topic": topic,
                }
            )
    if not new_rows:
        return []

    chunk_df = pd.concat([chunk_df, pd.DataFrame(new_rows)], ignore_index=True)
    affected_clusters = sorted({row["cluster"] for row in new_rows})
    for cluster_id in affected_clusters:
        _summarize_cluster(cluster_id)
    return affected_clusters


### Beispiel: Neues Dokument hinzuf?gen

Wir f?gen einen zus?tzlichen Programmier-Abschnitt hinzu. Nur der zugeh?rige Cluster wird aktualisiert; alle anderen bleiben unver?ndert.

In [ ]:
example_update = """Programming languages evolve quickly, and teams often need to add changelog entries without rebuilding their knowledge graphs. By tracking cluster centroids and summaries incrementally, we can capture these updates while keeping prior context intact."""
affected = add_texts_incrementally([example_update], source_label="incremental-demo")
print("Aktualisierte Cluster:", affected)
summary_df[summary_df["cluster"].isin(affected)]

## 12. Themen?bersicht & Retrieval

Damit wir jederzeit _?Zu welchen Themen hast du Informationen??_ beantworten k?nnen, lesen wir die Themen aus den Clusterzusammenfassungen und ? falls gew?nscht ? f?hren eine schnelle ?hnlichkeitssuche ?ber Chroma aus.

In [ ]:
def list_available_topics() -> list[str]:
    topics = sorted({topic for topic in topic_lookup.values() if topic})
    for topic in topics:
        print(f"- {topic}")
    return topics

available_topics = list_available_topics()
print(f"
Anzahl Topics: {len(available_topics)}")

In [ ]:
query = "Zu welchen Themen hast du Informationen?"
results = vector_store.similarity_search(query, k=3)
for doc in results:
    print(f"Topic: {doc.metadata.get('topic')} | Source: {doc.metadata.get('source')}")
    print(doc.page_content[:200].strip(), "...
")